In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch
import matplotlib.pyplot as plt
import pickle5 as pickle
import plotly.express as px
import itertools
import argparse
import scipy.stats
import scipy.special as special
from typing import Dict, List, Any, Tuple

from subset_utils import *

In [2]:
seed_everything(42)

In [3]:
os.getcwd()

'/scratch/users/oince22/hpc_run/data-cartography-for-compositionality/code/dataset_gen/transformer_generalization/cartography'

In [4]:
for i in dnames_cepochs():
    print(i)

('1685001853/cfq', 20)
('3960970220/cfq', 20)
('2895201892/cfq', 20)
('83256541/cogs', 10)
('4190663204/cogs', 10)
('3926193344/cogs', 10)
('42/pcfg', 140)


In [5]:
DNAMES_N_CEPOCHS = dnames_cepochs()
DNAMES_N_CEPOCHS = [t for t in DNAMES_N_CEPOCHS if "pcfg" in t[0]]
DNAMES_N_CEPOCHS

[('42/pcfg', 140)]

In [6]:
METRICS = ["Inverse PPL"] # "BLEU", "CHIA"
CRITERIA = ["Hard to Learn", "Ambiguous", "Easy to Learn", "Random"] # 
RATIOS = [0.5] # 0.33, 

for DATASET_NAME, CONVERGE_EPOCH in DNAMES_N_CEPOCHS:
    OUTPUTS_PATH = outputs_path(DATASET_NAME)
    idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE, min_epoch=3)
    df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
    for METRIC in METRICS:
        for CRITERION in CRITERIA:
            if CRITERION == "Random" and METRIC != "Inverse PPL":
                    continue
                    
            for RATIO in RATIOS:
                idx_fname = create_ratio_fname(METRIC, CRITERION, CONVERGE_EPOCH, RATIO)
                subset_df = choose_subset(df, METRIC, CRITERION, DATASET_NAME, idx_fname, ratio=RATIO)

Starting reading of training dynamics...


100%|██████████| 1169184/1169184 [34:30<00:00, 564.61it/s]


Finished reading of training dynamics.
Starting processing scores for Pandas DataFrames...


100%|██████████| 9353472/9353472 [02:31<00:00, 61886.31it/s] 


Finished processing scores for Pandas DataFrames.
Further processing of examples for length and rarity stats...


...

subset_idx: 41084


  0%|          | 0/41084 [00:00<?, ?it/s]


subset_idx: 41084


  0%|          | 0/41084 [00:00<?, ?it/s]


subset_idx: 41084


In [5]:
DNAMES = ["cfq", "cogs"]
METRICS = ["BLEU", "Inverse PPL", "CHIA"]
CRITERIA = ["Hard to Learn", "Ambiguous", "Easy to Learn", "Random"]
RATIOS = [0.33, 0.5]

run_commands = []

for DATASET_NAME in DNAMES:
    CONVERGE_EPOCH = DATASET2CEPOCHS[DATASET_NAME]
    OUTPUTS_PATH = outputs_path(DATASET_NAME)
    for METRIC in METRICS:
        for CRITERION in CRITERIA:
            if CRITERION == "Random" and METRIC != "Inverse PPL":
                continue
            
            for RATIO in RATIOS:
                indices_paths = []
                for SEED in DATASET2SEEDS[DATASET_NAME]:
                    indices_paths.append(wandb_config_name(SEED, DATASET_NAME, METRIC, CRITERION, CONVERGE_EPOCH, RATIO))                

                command = create_wandb_config(DATASET_NAME, METRIC, CRITERION, CONVERGE_EPOCH, RATIO, indices_paths)
                run_commands.append(command)

In [8]:
with open("../sweeps/commands/single_criterion_runs.txt", "w") as f:
    for command in run_commands:
        f.write(f"{command}\n")